## Top Ten Most Frequent Words in Star Trek: The Next Generation

The following script finds the ten most frequent words in Star Trek: The Next Generation. Many times top ten analyses are used to find the top ten words overall, but this particular script strips out stopwords so the resulting top ten words shed light on the top ten thematic words in the text.


### Global parameters 

You will need to have set up a [Karst account](https://kb.iu.edu/d/bezu#account) first. Once you have your Karst account and have set up access to Karst Desktop through thinlinc (see [textPrep.ipynb](https://github.com/cyberdh/Text-Analysis/blob/master/RNotebooks/textPrep.ipynb)), login to KD through the thinlinc client and go to Applications>Analytics>rstudio to launch R Studio.  Next, set the working directory by pointing to the location on Karst where you have stored the files. Below, we have chosen to save the folder "Text-Analysis" as a "Project" in R Studio on the Karst super-computer here at Indiana University. It contains the R scripts, texts, notebooks, and results. If you have forked and cloned the Github repository (see [textPrep.ipynb](https://github.com/cyberdh/Text-Analysis/blob/master/RNotebooks/textPrep.ipynb) for directions on how), simply point to where you have saved the folder. If you save it to your personal Karst folder, it will most likely look very similar to the example below. Karst is a unix server and so the home directory is represented by a ~ and, thus, the path will look like this "~/Text-Analysis/" (with the quotes). Alternatively, if you are on a PC, you will need to use an absolute path such as "C:/Users/XXX" (with the quotes again).

In R Studio, click Session in the menu bar > Set Working Directory > Choose Directory, then select the Text-Analysis directory in which you are working. This will set your working directory in the console pane, but make sure to copy the path into the source pane above to keep the directory constant if you close this script and reopen later. Make sure you click on the blue cube with a "R" in the center to set your working directory to your Text-Analysis project path.

HINT: Your working directory is the folder from which you will be pulling your texts.

In [1]:
setwd("~/Text-Analysis")

### Include necessary packages for notebook 

R's extensibility comes in large part from packages. Packages are groups of functions, data, and algorithms that allow users to easily carry out processes without recreating the wheel. Some packages are included in the basic installation of R, others created by R users are available for download. Make sure to have the following packages installed before beginning so that they can be accessed while running the scripts.

In R Studio, packages can be installed by navigating to Tools in the menu bar > Install Packages. Or in the bottom right panel click on the "packages" tab and then click on "install."

These packages are used within the top ten frequency script:

- NLP: Basic classes and methods for Natural Language Processing.

- tm: This package provides tools (functions) for performing various types of text mining. In this script, we will use tm to - performing text cleaning in order to have uniform data for analysis. Check out [this link](https://cran.r-project.org/web/packages/tm/vignettes/tm.pdf) for the documentation!

In [2]:
library(NLP)
library(tm)

### Read in an entire directory

The "Corpus" function from tm reads in the directory and saves it as a Corpus object. Here we are reading in every episode of Star Trek: The Next Generation.

In [3]:
corpus <- VCorpus(DirSource("data/StarTrekNextGenClean/series"))

### Scan in a single text

If you wish to only look at a single file or text, this code uses R's "scan" function to read in the text and is then saved as a variable called "text_raw"

In [4]:
#text_raw<-scan("data/StarTrekNextGenClean/series/277.txt", what="character", sep="\n")

### Save the text as a Corpus object

The "Corpus" function from tm reads in the single vector we scanned in the previous step and saves it as a Corpus object just as it does when reading in an entire directory.

In [5]:
#corpus <- VCorpus(VectorSource(text_raw))

### Clean the corpus

To clean the corpus in this example, we are using the tm package's built in tools to:
1. Lowercase all words so that "Lake" and "lake" are recognized as the same string.
2. Since we are working with Star Trek: The Next Generation, and Picard loves to quote Shakespeare, we scan in a pre-made early modern stopword list by scaning the text in as a character object and note that you wish to separate the text by line ("\n"). 
3. Remove stopwords using tm's built in english stopwords as well as the early modern stopwords list created by us at Cyber DH and any words that are not thematically relevant.
4. Remove punctuation as it is not germane to this analysis.
5. Strip out any whitespace between words to get rid of empty elements.

In [6]:
corpus <- tm_map(corpus, content_transformer(tolower))
#Add early modern stopwords
myStopWords <- scan("data/earlyModernStopword.txt", what="character", sep="\n")
corpus <- tm_map(corpus, removeWords, c(stopwords("english"), myStopWords, "just"))
corpus <- tm_map(corpus, removePunctuation)
corpus <- tm_map(corpus, stripWhitespace)

### Processing

After the corpus has been cleaned, we can now begin to process the text. The next step uses the DocumentTermMatrix() function again from the tm package. This function creates a matrix where each document is a row and the terms in the text make up the columns, saved here as "dtm." The rest of the matrix consists of the frequencies for each term within each document.

In [7]:
dtm <- DocumentTermMatrix(corpus)

Once we've created the Document-Term Matrix, we need to find the overall frequency of each term across the corpus. Here, we get the sums of the columns (colSums) within the Document-Term Matrix and save it as a sorted numeric vector called "freq."

In [8]:
freq <- sort(colSums(as.matrix(dtm)), decreasing = TRUE)

Now we turn the frequency matrix into a data frame

In [9]:
#Convert to data frame (helps make plot labeling easier)
freqDF <- as.data.frame.character(freq)

Now let's see what our data frame looks like

In [10]:
head(freqDF)

,freq
captain,4329
know,2812
data,2145
yes,2023
commander,1948
time,1944


We need to add a column that lists the words for each frequency. This line says that we want the rownames to be put in a column and that we want that column to have the header 'words'.

In [11]:
freqDF$words <- rownames(freqDF)

Now let's check our data frame again and see if it added our new column.

In [12]:
head(freqDF)

,freq,words
captain,4329,captain
know,2812,know
data,2145,data
yes,2023,yes
commander,1948,commander
time,1944,time


It did! Now, we are creating a variable that will determine the x-axis tick labels when we plot our bar graph. Make sure the number matches the number in the part of the plot below that says "barplot(head(freq,10)"

In [13]:
labNames <- head(freqDF$words, 10)

### Results

Finally, we can plot and view the data as a bar graph with the top ten words along the x-axis and the frequency of appearance along the y-axis. 

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5


In [14]:
par(mar=c(15,4,4,0))
x <- barplot(head(freq, 10), ylim=c(0,5000), col=c("red3", "orange3","yellow3","green3","blue3","darkorchid3","darkred", "darkorange", "gold", "darkgreen"), col.main="Gold", col.lab="red",
     main="Star Trek: The Next Generation", xlab="", ylab="Number of Occurences", xaxt="n")
axis(2, at=x, labels=TRUE)
axis(2, col="gray27", col.axis="gray27", col.ticks="gray27",labels=TRUE)
text(x, par("usr")[3], srt=45, adj=c(1.1,1.5),xpd = TRUE, labels = labNames, cex=.9, col="gray27", col.axis="gray27")
title(xlab='Top Ten Words', line=5, col.lab="red")
print(x)           

### VOILA!